# STAC Tutorial: Search and Visualize

This notebook shows how to display STAC GeoJSON foorprints and XYZ Tiles from assets on an interactive map using [Folium](https://python-visualization.github.io/folium/index.html). [pystac-client](https://pystac-client.readthedocs.io/) is used for searching STAC APIs. An external [TiTiler](https://github.com/developmentseed/titiler) service is used as an XYZ tiler.

The notebook starts with selecting an area of interest (several are provided in the "../aois" directory) and displaying the AOI on a map. This is followed by searching a STAC API and displaying the footprints on the same map. Finally, the assets will be added to the map by utilizing an external tiler to read the assets directly.

# Area of Interest

In [ ]:
# AOIs available

from glob import glob
from pprint import pprint

pprint(glob("../aois/*"))

In [ ]:
# read the GeoJSON file

import json
with open('../aois/bay-of-fundy.geojson', 'r') as f:
    aoi = json.load(f)
pprint(aoi)

In [ ]:
# use folium to display vectors
# Several folium basemap tiles are available:
#   - OpenStreetMap
#   - Stamen Terrain
#   - Stamen Toner
#   - Stamen Watercolor
#   - CartoDB positron
#   - CartoDB dark_matter

import folium

map = folium.Map(tiles='OpenStreetMap')

# add vector to map, as transparent polygon
folium.GeoJson(aoi, style_function = lambda x: {'fillColor': '#00000000'}).add_to(map)

map

In [ ]:
# fit the map to the bounds of the data

lons = [x[0] for x in aoi["geometry"]["coordinates"][0]]
lats = [x[1] for x in aoi["geometry"]["coordinates"][0]]
map.fit_bounds([(min(lats), min(lons)), (max(lats), max(lons))])
map

# Searching a STAC API

In [ ]:
APIS = {
    "earth-search": "https://earth-search.aws.element84.com/v1/",
    "planetary-computer": "https://planetarycomputer.microsoft.com/api/stac/v1",
    "usgs-landsat": "https://landsatlook.usgs.gov/stac-server",
    "nasa-lpdaac": "https://cmr.earthdata.nasa.gov/stac/LPCLOUD"
}

In [ ]:
# Use pystac-client to find data in the STAC API.

from pystac_client import Client
api = Client.open(APIS['earth-search'])
api

In [ ]:
# list collections

for collection in api.get_collections():
    print(f"{collection.id} - {collection.title}")

In [ ]:
# print collection
import pandas as pd

col = 'sentinel-2-l2a'

collection = api.get_collection(col)
collection

In [ ]:
# Search the API for Items

query = api.search(
    collections=[col],
    intersects=aoi['geometry'],
    datetime="2023-09-01/2023-10-31",
    limit=100,
    query = [
        "eo:cloud_cover<5"
    ]
)
item_collection = query.item_collection()

print(f"Found: {len(item_collection):d} STAC Items")

In [ ]:
# view footprints
style = {
    'fillColor': '#00000000', # transparent
    'color': '#fc0f03',       # red
    'weight': 1
}

for item in item_collection:
    folium.GeoJson(item.to_dict(), style_function=lambda x: style).add_to(map)

map

In [ ]:
pd.DataFrame.from_dict(collection.to_dict()['item_assets'], orient='index')

# Asset Visualization

In [ ]:
import requests

# add asset to map
asset = 'visual'

# URL of an XYZ Tile service
TILER_URL = 'https://d1v1jsbqwtcqnl.cloudfront.net'

def create_item_tilelayer(item, asset):
    # add tile layer of this asset to map with a percentile stretch
    href = item.assets[asset].href
    stats = requests.get(f"{TILER_URL}/cog/statistics?url=" + href).json()

    # rescale using percentile stretch
    rescale_params = ''
    for b in ['b1', 'b2', 'b3']:
        rescale_params += f"&rescale={stats[b]['percentile_2']},{stats[b]['percentile_98']}"

    # create tile layer
    tileset_url = TILER_URL + "/cog/tiles/{z}/{x}/{y}?&url=" + href
    tile_layer = folium.TileLayer(tiles = tileset_url + rescale_params, attr=item.id)
    return tile_layer

# add asset for this single item
item = item_collection[0]
tile_layer = create_item_tilelayer(item, asset=asset)
tile_layer.add_to(map)

map

In [ ]:
# add asset for all items
for item in item_collection:
    tile_layer = create_item_tilelayer(item, asset=asset)
    tile_layer.add_to(map)

map